In [1]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [2]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
from shapely import wkt
import os
from keplergl import KeplerGl

In [3]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [19]:
# 2020
usa_df = pd.read_csv(r'data/dest_nycxUS_daily_01-09-2020.csv')
# alter file, convert geoid field to string
usa_df = usa_df.rename(columns={'dest_stco':'id'})
usa_df['id'] = usa_df['id'].str[1:]
list_date = usa_df.columns.to_list()
list_date = list_date[1:]  

dff_20 = pd.melt(usa_df, id_vars=['id'],value_vars = list_date)

dff_20['variable'] = pd.to_datetime(dff_20['variable'])
dff_20.head()

,id,variable,value
0,01001,2020-01-01,0.249131
1,01003,2020-01-01,3.487827
2,01005,2020-01-01,0.249131
3,01007,2020-01-01,0.000000
4,01009,2020-01-01,0.249131


In [20]:
# 2019
usa_df = pd.read_csv(r'data/dest_nycxUS_daily_01-09-2019.csv')
# alter file, convert geoid field to string
usa_df = usa_df.rename(columns={'dest_stco':'id'})
usa_df['id'] = usa_df['id'].str[1:]
list_date = usa_df.columns.to_list()
list_date = list_date[1:]  

dff_19 = pd.melt(usa_df, id_vars=['id'],value_vars = list_date)

dff['variable'] = pd.to_datetime(dff_19['variable'])
dff_19.head()

,id,variable,value
0,01001,1/1/19,0.182967
1,01003,1/1/19,0.914833
2,01005,1/1/19,0.000000
3,01007,1/1/19,0.000000
4,01009,1/1/19,0.000000


In [5]:
# read in counties GeoJSON
counties = r'data/shapefiles/UScounties.geojson'
counties = gpd.read_file(counties)
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"MULTIPOLYGON (((-89.18137 37.04630, -89.17938 ..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"MULTIPOLYGON (((-84.44266 38.28324, -84.44114 ..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"MULTIPOLYGON (((-86.94486 37.07341, -86.94346 ..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"MULTIPOLYGON (((-84.12662 37.64540, -84.12483 ..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"MULTIPOLYGON (((-83.98428 38.44549, -83.98246 ..."


In [6]:
counties['geometry'] = counties.centroid
counties = counties.set_geometry('geometry')
#counties.head()
counties.to_file("data/shapefiles/UScounties_centroid.geojson", driver="GeoJSON")

C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [7]:
# add independent lat/longs for KeplerGl
counties['lon'] = counties['geometry'].x
counties['lat'] = counties['geometry'].y
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,POINT (-84.21715 38.20673),-84.217151,38.206735
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,POINT (-86.68162 37.20728),-86.681623,37.207285
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,POINT (-83.96431 37.69244),-83.964311,37.692444
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,POINT (-83.69666 38.37012),-83.696656,38.370118


In [22]:
usa_centroid20 = pd.merge(counties, dff_20, left_on="GEOID", right_on="id", how="inner")
usa_centroid20.to_csv(r'data/usa_cntr_jansep_20.csv')
usa_centroid19 =  pd.merge(counties, dff_19, left_on="GEOID", right_on="id", how="inner")
usa_centroid19.to_csv(r'data/usa_cntr_jansep_19.csv')

In [23]:
usa_centroid20.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat,id,variable,value
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-01,0.0
1,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-02,0.0
2,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-03,0.0
3,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-04,0.0
4,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-05,0.0


In [24]:
array = ['New York', 'Kings','Queens','Richmond','Bronx']
nyc = usa_centroid20.loc[(usa_centroid['STATEFP'] == '36') & usa_centroid['NAME'].isin(array)]
#usa_centroid = usa_centroid.loc[(usa_centroid['STATEFP'] != '36') & (df['NAME'] != 'New York', 'Kings','Queens','Richmond','Bronx')]
nyc.head()

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [25]:
len(usa_centroid20)

881184

In [26]:
usa_centroid20.tail()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat,id,variable,value
881179,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-09-26,0.676789
881180,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-09-27,0.335697
881181,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-09-28,0.338560
881182,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-09-29,0.933248
881183,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-09-30,0.616261


In [27]:
# remoing the NYC counties 
nyc_geoid = ['36005','36047','36061','36081','36085']
usa_centroid20 = usa_centroid20[~usa_centroid20['GEOID'].isin(nyc_geoid)]
usa_centroid19 = usa_centroid19[~usa_centroid19['GEOID'].isin(nyc_geoid)]

print(len(usa_centroid20))
print(len(usa_centroid19))

881184
1173840


In [28]:
# Aleutian Islands gives odd geography -- let's just remove from analysis
# Get names of indexes for which column Age has value 30
indexNames_20 = usa_centroid20[usa_centroid20['NAME'] == 'Aleutians West' ].index
indexNames_19 = usa_centroid19[usa_centroid19['NAME'] == 'Aleutians West' ].index
# Delete these row indexes from dataFrame
usa_centroid20.drop(indexNames_20, inplace=True)
usa_centroid19.drop(indexNames_19, inplace=True)

print(len(usa_centroid20))
print(len(usa_centroid19))

880910
1173475


In [29]:
df_centroid = pd.read_csv(r'data/values_cntr_jansep.csv')
usa20 = 'usa20'
usa19 = 'usa19'
subplaces = 'subplaces'
decimals = 0
def kepler_func(df,geo_name):
    df['date'] = pd.to_datetime(df['variable'])
    df['date'] = df["date"].dt.strftime('%m/%d/%Y')
    df['datetime'] = df['date'].astype(str) + ' 0:00'
    #df_centroid.drop(columns = 'geometry')
    #df['geom'] = df['geom'].apply(wkt.loads)
    df['value'] = df['value'].astype(float)
    df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
    df['value'] = df['value'].astype(int) # remove decimal
    # selecting rows based on value being greater than 0
    df = df.loc[df['value'] >= 1] 
    #renaming tooltip columns for Kepler.gl hover
    df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Trips/Device'})
    #df_centroid = gpd.GeoDataFrame(df_centroid, geometry='geom')
    df.to_csv(f'data/mn_to_{geo_name}.csv', index = False)

C:\Users\steve\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
# read geographies into the function
kepler_func(df_centroid, subplaces)
kepler_func(usa_centroid20, usa20)
kepler_func(usa_centroid19, usa19)

In [13]:
config = {
'version': 'v1',
'config': {
    'mapState': {
        'latitude': 40.7137,
        'longitude': -73.9675,
        'zoom': 12.3
    }}}

In [14]:
map1 = KeplerGl(height = 300)
map1.add_data(data=df_centroid,name ='df')
map1.config = config

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


ValueError: Can't clean for JSON: <shapely.geometry.point.Point object at 0x000002908F9D4208>